In [ ]:
import pandas as pd


In [ ]:
df=pd.read_csv(
    r"DataAnalysisSolarEnergy\data\Pecém_Solcast_energy_2023_60min.CSV", 
    sep=";",
    skiprows=11,
    encoding='latin-1',
    index_col=0,
    decimal=","
    )

df

In [ ]:
df=df.rename(columns={"kW":"ENERGIA"})
df

df/=1000

df=df.applymap(lambda x:0 if x<0 else x)
df.describe()

df.index=pd.to_datetime(df.index,format="%d/%m/%y %H:%M")
df.index

df.info

In [ ]:
medias_mensais=df.groupby(df.index.month).mean()

medias_mensais.plot(figsize=(18,6),marker="o")

In [ ]:
medias_horarias=df.groupby(df.index.hour).mean()

medias_horarias.plot(figsize=(18,6), kind="bar", color="orange")